In [1]:
import sys
from pathlib import Path
# Add the src directory to Python path
sys.path.insert(0, str(Path("..").resolve() / "src/"))

In [2]:
import numpy as np
import pandas as pd
from quantmetrics.levy_models import GBM, CJD, LJD, VG
from quantmetrics.risk_neutral.market_price_of_risk import MarketPriceOfRisk
from quantmetrics.option_pricing import Option, OptionPricer


In [3]:
path = Path("..").resolve().parent

In [4]:
option_df = pd.read_csv(path / 'tmp_data/option_with_sentiment.csv', index_col='date', parse_dates=True)
cjd_params_df = pd.read_csv(path / 'tmp_data/cjd_mle_params.csv', index_col='date', parse_dates=True)
ljd_params_df = pd.read_csv(path / 'tmp_data/ljd_mle_params.csv', index_col='date', parse_dates=True)
vg_params_df = pd.read_csv(path / 'tmp_data/vg_mle_params.csv', index_col='date', parse_dates=True)
option_df["DTB3"] = option_df["DTB3"] / 365  # Convert annual interest rates to daily
df = option_df.copy()[["mid_price", "strike_price","Close", "DTB3", "days_to_maturity", "sentiment_score"]]
df = df.rename(columns={"mid_price": "C", "strike_price":"K", "Close": "S0", "DTB3": "r", "days_to_maturity": "T", "sentiment_score": "st"})
df["r"] = df["r"].ffill()
df = df.groupby(df.index).agg({
    'C': list,
    'K': list,
    'S0': 'first',
    'r': 'first',
    'T': 'first',
    'st': "first"
})


In [5]:
selected_dates = pd.to_datetime(["2019-09-04","2019-11-04","2020-03-03", "2020-04-03", "2020-06-04", "2020-08-04", "2020-10-06", "2020-12-04"])
date = selected_dates[0]

In [6]:
K = df.loc[date, 'K'][-1]

In [7]:
ljd = LJD(
    S0=df.loc[date, 'S0'],
    mu = ljd_params_df.loc[date, 'mu'],
    sigma = ljd_params_df.loc[date, 'sigma'],
    lambda_ = ljd_params_df.loc[date, 'lambda_'],
    muJ = ljd_params_df.loc[date, 'muJ'],
    sigmaJ = ljd_params_df.loc[date, 'sigmaJ']
)

vg = VG(
    S0=df.loc[date, 'S0'],
    mu=vg_params_df.loc[date, 'mu'],
    m=vg_params_df.loc[date, 'm'],
    delta=vg_params_df.loc[date, 'delta'],
    kappa=vg_params_df.loc[date, 'kappa'],
)

psi = np.linspace(-10000, 2000, 300)
ljd_thetas_exact = np.zeros(psi.shape, dtype=float)
ljd_thetas_levy = np.zeros(psi.shape, dtype=float)
vg_thetas_levy = np.zeros(psi.shape, dtype=float)



for i in range(len(psi)):
    option = Option(
            K=K,
            r=df.loc[date, 'r'],
            T=df.loc[date, 'T'],
            q=0,
            emm = "Esscher",
            psi = psi[i],
        )
    ljd_exact_mpr = MarketPriceOfRisk(ljd, option)
    ljd_levy_mpr = MarketPriceOfRisk(ljd, option)
    vg_levy_mpr = MarketPriceOfRisk(vg, option)

    ljd_thetas_exact[i] = ljd_exact_mpr.solve(
        exact=True,
        M=2.0,
        N_center=150,
        N_tails=100,
        EXP_CLIP=700,
        search_bounds = [-50, 50],
        xtol=1e-8,
        rtol=1e-8,
        maxiter=500)
    
    ljd_thetas_levy[i] = ljd_levy_mpr.solve(
        exact=False,
        M=2.0,
        N_center=150,
        N_tails=100,
        EXP_CLIP=700,
        search_bounds = [-50, 50],
        xtol=1e-8,
        rtol=1e-8,
        maxiter=500)
    
    vg_thetas_levy[i] = vg_levy_mpr.solve(
        exact=False,
        M=2.0,
        N_center=150,
        N_tails=100,
        EXP_CLIP=700,
        search_bounds = [-100, 100],
        xtol=1e-8,
        rtol=1e-8,
        maxiter=500)

df_thetas_test = pd.DataFrame({"psi": psi,  "ljd_theta_exact": ljd_thetas_exact, "ljd_theta_levy": ljd_thetas_levy, "vg_theta":vg_thetas_levy})

KeyboardInterrupt: 

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create subplot figure with 2 rows
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,  # Smaller value = closer plots
    subplot_titles=('VG θ(ψ)', 'LJD θ(ψ) approx', 'LJD θ(ψ) exact')
)

# ψ(t) trace (top plot)
fig.add_trace(go.Scatter(
    x=df_thetas_test['psi'],
    y=df_thetas_test['vg_theta'],
    name='VG θ(ψ)',
    line=dict(color='purple'),
    opacity=0.8
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_thetas_test['psi'],
    y=df_thetas_test['ljd_theta_levy'],
    name='LJD θ(ψ) approx',
    line=dict(color='mediumblue'),
    opacity=0.8
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=df_thetas_test['psi'],
    y=df_thetas_test['ljd_theta_exact'],
    name='LJD θ(ψ) exact',
    line=dict(color='blue'),
    opacity=0.8
), row=3, col=1)

# Update layout
fig.update_layout(
    height=600,
    width=1000,
    title=f'Date: {date.date()} and K/S0 = {K/df.loc[date,"S0"]}',
    template='plotly_white',
    legend=dict(x=0.95, y=1.2)
)

# Customize y-axis fonts
fig.update_yaxes(title_text='VG', tickfont=dict(color='purple'), row=1, col=1)
fig.update_yaxes(title_text='LJD approx', tickfont=dict(color='mediumblue'), row=2, col=1)
fig.update_yaxes(title_text='LJD exact', tickfont=dict(color='blue'), row=3, col=1)

# Customize x-axis
fig.update_xaxes(title_text='ψ', tickangle=45, nticks=12, row=3, col=1)

fig.show()

In [7]:
from quantmetrics.price_calculators.characteristic_function import CharacteristicFunction

In [8]:

g = 2  # factor to increase accuracy
N = g * 2**3
eps = (g * 150.0) ** -1
eta = 2 * np.pi / (N * eps)
u = np.arange(1, N + 1, 1)
vo = eta * (u - 1)
alpha_itm = 1.5

omega = vo - (alpha_itm + 1) * 1j


ljd_levy = LJD(
    S0=df.loc[date, 'S0'],
    mu = ljd_params_df.loc[date, 'mu'],
    sigma = ljd_params_df.loc[date, 'sigma'],
    lambda_ = ljd_params_df.loc[date, 'lambda_'],
    muJ = ljd_params_df.loc[date, 'muJ'],
    sigmaJ = ljd_params_df.loc[date, 'sigmaJ']
)
vg = VG(
    S0=df.loc[date, 'S0'],
    mu=vg_params_df.loc[date, 'mu'],
    m=vg_params_df.loc[date, 'm'],
    delta=vg_params_df.loc[date, 'delta'],
    kappa=vg_params_df.loc[date, 'kappa'],
)

option = Option(
            K=K,
            r=df.loc[date, 'r'],
            T=df.loc[date, 'T'],
            q=0,
            emm = "Esscher",
            psi = 0,
        )
u = omega # np.arange(10,100,10) #omega

char_ljd = CharacteristicFunction(ljd_levy,option)

char_vg = CharacteristicFunction(vg, option)

ljd_int_exact = char_ljd(u, exact = True)
ljd_int_levy = char_ljd(u, exact = False, M_int = .5, N_int = 300) # Converges much faster than the VG

vg_int_exact = char_vg(u, exact = True, search_bounds=(-100,100))
vg_int_levy = char_vg(u, exact = False, M_int = 1, N_int = 20000, search_bounds=(-100,100)) # Needs more points to converge, 100,000 at least for better accuracy
df_ljd_int_tmp = pd.DataFrame({"u":u, "LJD_chr_exact":ljd_int_exact, "LJD_chr_levy":ljd_int_levy, "VG_chr_exact":vg_int_exact, "VG_chr_levy":vg_int_levy})
df_ljd_int_tmp


,u,LJD_chr_exact,LJD_chr_levy,VG_chr_exact,VG_chr_levy
0,0.000000- 2.500000j,1.008387e+00+0.000000e+ 00j,1.008387e+00+0.000000e+ 00j,1.007312e+00+0.000000e+ 00j,1.007313e+00+0.000000e+ 00j
1,117.809725- 2.500000j,-7.469727e-06-7.170288e- 06j,-7.469727e-06-7.170288e- 06j,7.507698e-07+4.497258e- 05j,7.491895e-07+4.498263e- 05j
2,235.619449- 2.500000j,-1.885749e-13-1.128012e- 13j,-1.885749e-13-1.128012e- 13j,-1.206427e-12-1.060178e- 11j,-1.206675e-12-1.061130e- 11j
3,353.429174- 2.500000j,-6.095165e-25+5.199840e- 25j,-6.095165e-25+5.199840e- 25j,3.221655e-18+4.191542e- 17j,3.223203e-18+4.199962e- 17j
4,471.238898- 2.500000j,3.250569e-41+7.317274e- 41j,3.250569e-41+7.317274e- 41j,-2.569591e-22-2.281564e- 21j,-2.575174e-22-2.289720e- 21j
5,589.048623- 2.500000j,2.128172e-61-2.779280e- 62j,2.128172e-61-2.779280e- 62j,1.628624e-25+7.785203e- 25j,1.636164e-25+7.828836e- 25j
6,706.858347- 2.500000j,2.432950e-87-1.523795e- 86j,2.432950e-87-1.523795e- 86j,-3.364364e-28-9.565606e- 28j,-3.389103e-28-9.643125e- 28j
7,824.668072- 2.500000j,-2.684023e-116-1.285013e- 116j,-2.684023e-116-1.285013e- 116j,1.591069e-30+2.948885e- 30j,1.607654e-30+2.981564e- 30j
8,942.477796- 2.500000j,-1.031841e-150+1.142148e- 150j,-1.031841e-150+1.142148e- 150j,-1.410740e-32-1.788337e- 32j,-1.430356e-32-1.814363e- 32j
9,1060.287521- 2.500000j,1.112374e-189+1.822879e- 189j,1.112374e-189+1.822879e- 189j,2.047614e-34+1.789445e- 34j,2.084117e-34+1.822631e- 34j


In [9]:

g = 2  # factor to increase accuracy
N = g * 2**3
eps = (g * 150.0) ** -1
eta = 2 * np.pi / (N * eps)
u = np.arange(1, N + 1, 1)
vo = eta * (u - 1)
alpha_itm = 1.5

omega = vo - (alpha_itm + 1) * 1j


ljd_levy = LJD(
    S0=df.loc[date, 'S0'],
    mu = ljd_params_df.loc[date, 'mu'],
    sigma = ljd_params_df.loc[date, 'sigma'],
    lambda_ = ljd_params_df.loc[date, 'lambda_'],
    muJ = ljd_params_df.loc[date, 'muJ'],
    sigmaJ = ljd_params_df.loc[date, 'sigmaJ']
)
vg = VG(
    S0=df.loc[date, 'S0'],
    mu=vg_params_df.loc[date, 'mu'],
    m=vg_params_df.loc[date, 'm'],
    delta=vg_params_df.loc[date, 'delta'],
    kappa=vg_params_df.loc[date, 'kappa'],
)

option = Option(
            K=K,
            r=df.loc[date, 'r'],
            T=df.loc[date, 'T'],
            q=0,
            emm = "Esscher",
            psi = 20,
        )
u = omega # np.arange(10,100,10) #omega

char_ljd = CharacteristicFunction(ljd_levy,option)

char_vg = CharacteristicFunction(vg, option)

ljd_int_exact = char_ljd(u, exact = True)
ljd_int_levy = char_ljd(u, exact = False, M_int = 1.0, N_int = 300)


vg_int_levy = char_vg(u, exact = False, M_int = 1.0, N_int = 20_000)
df_ljd_int_tmp = pd.DataFrame({"u":u, "LJD_chr_exact":ljd_int_exact, "LJD_chr_levy":ljd_int_levy,  "VG_chr_levy":vg_int_levy})
df_ljd_int_tmp


,u,LJD_chr_exact,LJD_chr_levy,VG_chr_levy
0,0.000000- 2.500000j,1.008420e+00+0.000000e+ 00j,1.008420e+00+0.000000e+ 00j,1.007335e+00+0.000000e+ 00j
1,117.809725- 2.500000j,-7.010661e-06-7.064504e- 06j,-7.010661e-06-7.064504e- 06j,4.013764e-07+4.353350e- 05j
2,235.619449- 2.500000j,-1.794481e-13-1.159798e- 13j,-1.794481e-13-1.159798e- 13j,-1.070342e-12-1.019199e- 11j
3,353.429174- 2.500000j,-6.176920e-25+4.756994e- 25j,-6.176920e-25+4.756994e- 25j,2.817704e-18+4.035717e- 17j
4,471.238898- 2.500000j,2.681440e-41+7.315518e- 41j,2.681440e-41+7.315518e- 41j,-2.354959e-22-2.202859e- 21j
5,589.048623- 2.500000j,2.086270e-61-9.701639e- 63j,2.086276e-61-9.689813e- 63j,1.540262e-25+7.539951e- 25j
6,706.858347- 2.500000j,3.837043e-87-1.451745e- 86j,4.448372e-87-1.467362e- 86j,-3.227069e-28-9.295451e- 28j
7,824.668072- 2.500000j,-2.448406e-116-1.546177e- 116j,1.508109e-116-1.907351e- 115j,1.538624e-30+2.876235e- 30j
8,942.477796- 2.500000j,-1.143022e-150+9.679800e- 151j,-3.963002e-147+3.356111e- 147j,-1.372598e-32-1.751539e- 32j
9,1060.287521- 2.500000j,8.050234e-190+1.915782e- 189j,1.367886e-188+1.186938e- 189j,2.003117e-34+1.760974e- 34j


In [12]:
# Use the original code to compare prices
ljd = LJD(
    S0=df.loc[date, 'S0'],
    mu = ljd_params_df.loc[date, 'mu'],
    sigma = ljd_params_df.loc[date, 'sigma'],
    lambda_ = ljd_params_df.loc[date, 'lambda_'],
    muJ = ljd_params_df.loc[date, 'muJ'],
    sigmaJ = ljd_params_df.loc[date, 'sigmaJ']
)

vg = VG(
    S0=df.loc[date, 'S0'],
    mu=vg_params_df.loc[date, 'mu'],
    m=vg_params_df.loc[date, 'm'],
    delta=vg_params_df.loc[date, 'delta'],
    kappa=vg_params_df.loc[date, 'kappa'],
)

option = Option(
            K=df.loc[date, 'K'],
            r=df.loc[date, 'r'],
            T=df.loc[date, 'T'],
            q=0,
            emm = "Esscher",
            psi = 0,
        )

ljd_pricer_exact = OptionPricer(ljd, option)
ljd_prices_exact = ljd_pricer_exact.fft(exact = True, L = 1e-12, search_bounds=(-100,100))[0]

ljd_pricer_levy = OptionPricer(ljd, option)
ljd_prices_levy = ljd_pricer_levy.fft(exact=False, M_int = 1,L = 1e-12, search_bounds=(-100,100))[0]

vg_pricer_exact = OptionPricer(vg, option)
vg_prices_exact = vg_pricer_exact.fft(exact=True,L = 1e-12, search_bounds=(-100,100))[0]

vg_pricer_levy = OptionPricer(vg, option)
vg_prices_levy = vg_pricer_levy.fft(exact=False, M_int = 1.0, N_int=20_000,L = 1e-12, search_bounds=(-100,100))[0]

df_option_prices_test = pd.DataFrame({
    "K": option.K,
    "LJD exact": ljd_prices_exact,
    "LJD approx": ljd_prices_levy,
    "VG exact": vg_prices_exact,
    "VG approx": vg_prices_levy
})

df_option_prices_test

,K,LJD exact,LJD approx,VG exact,VG approx
0,2925.0,67.201470,67.201470,59.932411,59.932640
1,2970.0,45.267017,45.267017,38.358488,38.358550
2,3020.0,27.267373,27.267373,21.548750,21.548703
3,2835.0,126.991551,126.991551,121.261619,121.262274
4,2840.0,123.171018,123.171018,117.296458,117.297090
5,2885.0,91.300745,91.300745,84.387805,84.388219
6,2900.0,81.770659,81.770659,74.651935,74.652277
7,2915.0,72.828198,72.828198,65.589247,65.589520
8,2925.0,67.201470,67.201470,59.932411,59.932640
9,2950.0,54.329335,54.329335,47.162901,47.163030


In [13]:
date

Timestamp('2019-09-04 00:00:00')

In [19]:
vg_calibration = pd.DataFrame(
    np.nan,
    index = [date],#selected_dates,
    columns = ["theta_1stEss", "rmse_1stEss","psi_cal", "theta_cal", "rmse_cal"]
    )

unsuccessful_dates_vg = []

In [20]:
from tqdm import tqdm
from scipy.optimize import  minimize_scalar
# Regularization strength
lambda_reg = 1e-8

# Search bounds
psi_lower_limit = -15000
psi_upper_limit = 0

# optimizer tolerence
TOL = 1e-8

for date in tqdm([date], desc="Calibrating"):
    try:
        iter_count = 0
        min_error = float("inf")

        def rmse_error(psi):
            global iter_count, min_error

            # Check bounds before proceeding
            #if not (psi < psi_upper_bound):
                #return np.inf

            try:
                vg = VG(
                    S0=df.loc[date, 'S0'],
                    mu=vg_params_df.loc[date, 'mu'],
                    m=vg_params_df.loc[date, 'm'],
                    delta=vg_params_df.loc[date, 'delta'],
                    kappa=vg_params_df.loc[date, 'kappa'],
                )
                K = np.array(df.loc[date, "K"])
                option = Option(
                        r=df.loc[date, "r"],
                        q = 0,
                        K=K,
                        T=df.loc[date, "T"],
                        emm='Esscher',
                        psi=psi
                )

                pricer = OptionPricer(vg, option)
                    
                model_prices = pricer.fft(exact=False, M_int = 1.0, N_int=20_000)[0]

                # Log and catch invalid model outputs
                if np.any(np.isnan(model_prices)) or np.any(np.isinf(model_prices)):
                    #print(f"⚠️ Invalid model_prices at psi={psi:.4f} on {date.date()}")
                    return np.inf

                market_prices = np.array(df.loc[date, "C"])

                # Log and catch invalid market data
                if np.any(np.isnan(market_prices)) or np.any(np.isinf(market_prices)):
                    print(f"⚠️ Invalid market_prices on {date.date()}")
                    return np.inf

                # Optionally clamp extreme values
                # model_prices = np.clip(model_prices, 1e-6, 1e6)
                # market_prices = np.clip(market_prices, 1e-6, 1e6)
                rmse = np.sqrt(np.mean((model_prices - market_prices) ** 2))
                l2_penalty = lambda_reg * psi**2

                iter_count += 1
                min_error = min(min_error, rmse)

                return rmse + l2_penalty

            except Exception as inner_e:
                print(f"Inner error during pricing at psi={psi:.2f}: {inner_e}")
                return np.inf

        result = minimize_scalar(
            rmse_error,
            method='bounded',
            bounds=(psi_lower_limit, psi_upper_limit),
            options={'xatol': TOL}
        )

        #print(f"Optimization result for {date.date()}: {result}")

        # Validate result BEFORE saving
        if not result.success or np.isnan(result.fun) or np.isinf(result.fun):
            print(f"⚠️ {date.date()} — Optimization failed or RMSE invalid.")
            unsuccessful_dates_vg.append(date)

            # Optional retry strategy
            # print("🔄 Retrying with tighter bounds…")
            # result_retry = minimize_scalar(rmse_error, bounds=(-500, min(psi_upper_bound, 250)), method='bounded')
            # if result_retry.success and not np.isinf(result_retry.fun):
            #     result = result_retry
            # else:
            continue

        # Calculate thetas
        vg = VG(
                S0=df.loc[date, 'S0'],
                mu=vg_params_df.loc[date, 'mu'],
                m=vg_params_df.loc[date, 'm'],
                delta=vg_params_df.loc[date, 'delta'],
                kappa=vg_params_df.loc[date, 'kappa'],
                )
        option_cal_psi = Option(
                r=df.loc[date, "r"],
                q=0,
                K=np.array(df.loc[date, "K"]),
                T=df.loc[date, "T"],
                emm='Esscher',
                psi=result.x
            )
        
        K = np.array(df.loc[date, "K"])
        #option_prices_1stEss = np.zeros(K.shape, dtype=float)
        
        
        option_1stEss = Option(
                r=df.loc[date, "r"],
                q=0,
                K=K,
                T=df.loc[date, "T"],
                emm='Esscher',
                psi=0
        )

        pricer_1stEss = OptionPricer(vg, option_1stEss)
        option_prices_1stEss, theta_1stEss = pricer_1stEss.fft(exact=True, M = 1, search_bounds = [-30,30])

        mpr_cal = MarketPriceOfRisk(vg, option_cal_psi)
        
        theta_cal = mpr_cal.solve(exact=False,
        M=2.0,
        N_center=150,
        N_tails=100,
        EXP_CLIP=700,
        search_bounds = [-100, 100],
        xtol=1e-8,
        rtol=1e-8,
        maxiter=500)

        
        # Extract market option the series
        market_option = df.loc[date, "C"]

        # RMSE of first order Esscher
        rmse_1stEss = np.sqrt(np.mean((option_prices_1stEss - market_option) ** 2))

        # Save results
        vg_calibration.loc[date, "theta_1stEss"] = theta_1stEss
        vg_calibration.loc[date, "rmse_1stEss"] = rmse_1stEss
        vg_calibration.loc[date, "psi_cal"] = result.x
        vg_calibration.loc[date, "theta_cal"] = theta_cal
        vg_calibration.loc[date, "rmse_cal"] = result.fun

        print(f"✅ {date.date()} — Success | psi = {result.x:.4f} | theta_cal = {theta_cal:.4f} | RMSE_cal = {result.fun:.6f} | theta_1stess = {theta_1stEss:.4f} | RMSE_1stEss = {rmse_1stEss:.4f} | Iter = {iter_count}")

        # Save every 10 successful steps
        #if len(vg_calibration.dropna()) % 10 == 0:
            #vg_calibration.to_csv(file_path_vg, index=True)

    except Exception as e:
        print(f"❌ Error on {date.date()} | Bounds: ({psi_lower_limit}, {psi_upper_limit:.2f}) — {e}")
        unsuccessful_dates_vg.append(date)
        continue

# code took 147 minutes to complete

Calibrating: 100%|██████████| 1/1 [14:01<00:00, 841.97s/it]

✅ 2019-09-04 — Success | psi = -1616.3994 | theta_cal = -11.9435 | RMSE_cal = 4.661159 | theta_1stess = -7.0866 | RMSE_1stEss = 7.5288 | Iter = 25


In [11]:
cjd_calibration = pd.DataFrame(
    np.nan,
    index = [date],#selected_dates,
    columns = ["theta_1stEss", "rmse_1stEss","psi_cal", "theta_cal", "rmse_cal"]
    )

unsuccessful_dates_cjd = []

In [12]:
from tqdm import tqdm
from scipy.optimize import  minimize_scalar
# Regularization strength
lambda_reg = 1e-8

# Search bounds
psi_lower_limit = -9000
psi_upper_limit = 15000

# optimizer tolerence
TOL = 1e-10

for date in tqdm([date], desc="Calibrating"):
    try:
        iter_count = 0
        min_error = float("inf")
        #sigmaJ = ljd_params_df.loc[date, 'sigmaJ']
        #ljd_psi_upper_bound = 1.0 / (2 * sigmaJ**2)
        def rmse_error(psi):
            global iter_count, min_error

            # Check bounds before proceeding
            #if not (psi < psi_upper_bound):
                #return np.inf

            try:
                cjd = CJD(
                S0=df.loc[date, 'S0'],
                mu=cjd_params_df.loc[date, 'mu'],
                sigma=cjd_params_df.loc[date, 'sigma'],
                lambda_=cjd_params_df.loc[date, 'lambda_'],
                gamma=cjd_params_df.loc[date, 'gamma'],
                )
                K = np.array(df.loc[date, "K"])
                option = Option(
                        r=df.loc[date, "r"],
                        q = 0,
                        K=K,
                        T=df.loc[date, "T"],
                        emm='Esscher',
                        psi=psi
                )

                pricer = OptionPricer(cjd, option)
                    
                model_prices = pricer.fft(
                    exact=True,
                    search_bounds=(-100,100),
                    xtol=1e-10,
                    rtol=1e-10)[0]

                # Log and catch invalid model outputs
                if np.any(np.isnan(model_prices)) or np.any(np.isinf(model_prices)):
                    #print(f"⚠️ Invalid model_prices at psi={psi:.4f} on {date.date()}")
                    return np.inf

                market_prices = np.array(df.loc[date, "C"])

                # Log and catch invalid market data
                if np.any(np.isnan(market_prices)) or np.any(np.isinf(market_prices)):
                    print(f"⚠️ Invalid market_prices on {date.date()}")
                    return np.inf

                # Optionally clamp extreme values
                # model_prices = np.clip(model_prices, 1e-6, 1e6)
                # market_prices = np.clip(market_prices, 1e-6, 1e6)
                rmse = np.sqrt(np.mean((model_prices - market_prices) ** 2))
                l2_penalty = lambda_reg * psi**2

                iter_count += 1
                min_error = min(min_error, rmse)

                return rmse + l2_penalty

            except Exception as inner_e:
                print(f"Inner error during pricing at psi={psi:.2f}: {inner_e}")
                return np.inf

        result = minimize_scalar(
            rmse_error,
            method='bounded',
            bounds=(psi_lower_limit, psi_upper_limit),
            options={'xatol': TOL}
        )

        #print(f"Optimization result for {date.date()}: {result}")

        # Validate result BEFORE saving
        if not result.success or np.isnan(result.fun) or np.isinf(result.fun):
            print(f"⚠️ {date.date()} — Optimization failed or RMSE invalid.")
            unsuccessful_dates_cjd.append(date)

            # Optional retry strategy
            # print("🔄 Retrying with tighter bounds…")
            # result_retry = minimize_scalar(rmse_error, bounds=(-500, min(psi_upper_bound, 250)), method='bounded')
            # if result_retry.success and not np.isinf(result_retry.fun):
            #     result = result_retry
            # else:
            continue

        # Calculate thetas
        cjd = CJD(
                S0=df.loc[date, 'S0'],
                mu=cjd_params_df.loc[date, 'mu'],
                sigma=cjd_params_df.loc[date, 'sigma'],
                lambda_=cjd_params_df.loc[date, 'lambda_'],
                gamma=cjd_params_df.loc[date, 'gamma'],
                )
        option_cal_psi = Option(
                r=df.loc[date, "r"],
                q=0,
                K=np.array(df.loc[date, "K"]),
                T=df.loc[date, "T"],
                emm='Esscher',
                psi=result.x
            )
        
        K = np.array(df.loc[date, "K"])
        #option_prices_1stEss = np.zeros(K.shape, dtype=float)
        
        
        option_1stEss = Option(
                r=df.loc[date, "r"],
                q=0,
                K=K,
                T=df.loc[date, "T"],
                emm='Esscher',
                psi=0
        )

        pricer_1stEss = OptionPricer(cjd, option_1stEss)
        option_prices_1stEss, theta_1stEss = pricer_1stEss.fft(
            exact=True,
            search_bounds = [-100,100])

        

        mpr_cal = MarketPriceOfRisk(cjd, option_cal_psi)
        
        theta_cal = mpr_cal.solve(
            L=12-16,
            exact=True,
            search_bounds = (-100, 100),
            xtol=1e-10,
            rtol=1e-10,
            )

        
        # Extract market option the series
        market_option = df.loc[date, "C"]

        # RMSE of first order Esscher
        rmse_1stEss = np.sqrt(np.mean((option_prices_1stEss - market_option) ** 2))

        # Save results
        cjd_calibration.loc[date, "theta_1stEss"] = theta_1stEss
        cjd_calibration.loc[date, "rmse_1stEss"] = rmse_1stEss
        cjd_calibration.loc[date, "psi_cal"] = result.x
        cjd_calibration.loc[date, "theta_cal"] = theta_cal
        cjd_calibration.loc[date, "rmse_cal"] = result.fun

        print(f"✅ {date.date()} — Success | psi = {result.x:.4f} | theta_cal = {theta_cal:.4f} | RMSE_cal = {result.fun:.6f} | theta_1stess = {theta_1stEss:.4f} | RMSE_1stEss = {rmse_1stEss:.4f} | Iter = {iter_count}")

        # Save every 10 successful steps
        #if len(vg_calibration.dropna()) % 10 == 0:
            #vg_calibration.to_csv(file_path_vg, index=True)

    except Exception as e:
        print(f"❌ Error on {date.date()} | Bounds: ({psi_lower_limit}, {psi_upper_limit:.2f}) — {e}")
        unsuccessful_dates_cjd.append(date)
        continue

# code took 147 minutes to complete

Calibrating:   0%|          | 0/1 [00:00<?, ?it/s]

Calibrating: 100%|██████████| 1/1 [00:00<00:00,  3.78it/s]

✅ 2019-09-04 — Success | psi = -5538.9721 | theta_cal = -32.4881 | RMSE_cal = 5.315888 | theta_1stess = -3.9400 | RMSE_1stEss = 12.3159 | Iter = 15


In [9]:
cjd = CJD(
    S0=df.loc[date, 'S0'],
    mu=cjd_params_df.loc[date, 'mu'],
    sigma=cjd_params_df.loc[date, 'sigma'],
    lambda_=cjd_params_df.loc[date, 'lambda_'],
    gamma=cjd_params_df.loc[date, 'gamma'],
)

option= Option(
    r=df.loc[date, "r"],
    q=0,
    K=np.array(df.loc[date, "K"]),
    T=df.loc[date, "T"],
    emm='Esscher',
    psi=-100
)
cjd_pricer = OptionPricer(cjd, option)

cjd_price = cjd_pricer.fft(exact=False, search_bounds=(-1e+10,1e+10))[0]
cjd_exact = cjd_pricer.closed_form()
market_price = df.loc[date, "C"]

pd.DataFrame({"K": np.array(df.loc[date,"K"]), "CJD_price": cjd_price, "Market": market_price, "closed_form": cjd_exact})

C:\Users\ellae\Projects\Active_projects\quantmetrics\src\quantmetrics\risk_calculators\martingale_equation.py:66: RuntimeWarning: overflow encountered in exp
  Exp = np.exp(x * gamma + psi * gamma**2)


,K,CJD_price,Market,closed_form
0,2925.0,67.108468,56.550,67.108470
1,2970.0,45.114618,29.350,45.114619
2,3020.0,27.026521,9.500,27.026522
3,2835.0,126.929564,127.550,126.929565
4,2840.0,123.109449,123.450,123.109450
5,2885.0,91.234047,87.900,91.234048
6,2900.0,81.696983,76.750,81.696985
7,2915.0,72.744135,66.050,72.744136
8,2925.0,67.108468,59.200,67.108470
9,2950.0,54.206951,43.200,54.206952


In [10]:
from quantmetrics.risk_calculators.martingale_equation import RiskPremium
(
    MarketPriceOfRisk(cjd, option).solve(exact=True, search_bounds = (-1e+10, 1e+10), xtol=1e-10, rtol=1e-10),
    RiskPremium(cjd, option).calculate()
)

C:\Users\ellae\Projects\Active_projects\quantmetrics\src\quantmetrics\risk_calculators\martingale_equation.py:66: RuntimeWarning: overflow encountered in exp
  Exp = np.exp(x * gamma + psi * gamma**2)


(-4.994480375062349, -4.994480375062343)